In [1]:
# Attempting to load and explore the dataset again
import pandas as pd

# Load the dataset
data_path = 'loan_data.csv'
loan_data = pd.read_csv(data_path)

# Display the first few rows of the dataset to understand its structure
loan_data.head(), loan_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 0 to 380
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            381 non-null    object 
 1   Gender             376 non-null    object 
 2   Married            381 non-null    object 
 3   Dependents         373 non-null    object 
 4   Education          381 non-null    object 
 5   Self_Employed      360 non-null    object 
 6   ApplicantIncome    381 non-null    int64  
 7   CoapplicantIncome  381 non-null    float64
 8   LoanAmount         381 non-null    float64
 9   Loan_Amount_Term   370 non-null    float64
 10  Credit_History     351 non-null    float64
 11  Property_Area      381 non-null    object 
 12  Loan_Status        381 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 38.8+ KB


(    Loan_ID Gender Married Dependents     Education Self_Employed  \
 0  LP001003   Male     Yes          1      Graduate            No   
 1  LP001005   Male     Yes          0      Graduate           Yes   
 2  LP001006   Male     Yes          0  Not Graduate            No   
 3  LP001008   Male      No          0      Graduate            No   
 4  LP001013   Male     Yes          0  Not Graduate            No   
 
    ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
 0             4583             1508.0       128.0             360.0   
 1             3000                0.0        66.0             360.0   
 2             2583             2358.0       120.0             360.0   
 3             6000                0.0       141.0             360.0   
 4             2333             1516.0        95.0             360.0   
 
    Credit_History Property_Area Loan_Status  
 0             1.0         Rural           N  
 1             1.0         Urban           Y  
 2 

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Drop the 'Loan_ID' column
loan_data = loan_data.drop(columns=['Loan_ID'])

# Separating features and target variable
X = loan_data.drop(columns='Loan_Status')
y = loan_data['Loan_Status'].apply(lambda x: 1 if x == 'Y' else 0)  # Encode target variable as binary

# Identify numerical and categorical columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

# Preprocessing for numerical data: imputation + scaling
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

# Preprocessing for categorical data: imputation + one-hot encoding
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Apply preprocessing
X_train_prepared = preprocessor.fit_transform(X_train)
X_test_prepared = preprocessor.transform(X_test)

X_train_prepared.shape, X_test_prepared.shape


((304, 20), (77, 20))

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_prepared.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [4]:
history = model.fit(X_train_prepared, y_train,
                    epochs=100,
                    validation_split=0.2,
                    batch_size=10,
                    verbose=1)


Epoch 1/100


C:\Users\18135\anaconda3\envs\Project1\lib\site-packages\keras\src\engine\data_adapter.py:1808: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]




25/25 [==============================] - 1s 11ms/step - loss: 0.5857 - accuracy: 0.7037 - val_loss: 0.4936 - val_accuracy: 0.7377
Epoch 2/100
25/25 [==============================] - 0s 3ms/step - loss: 0.4605 - accuracy: 0.8189 - val_loss: 0.4125 - val_accuracy: 0.8852
Epoch 3/100
25/25 [==============================] - 0s 4ms/step - loss: 0.3961 - accuracy: 0.8477 - val_loss: 0.4039 - val_accuracy: 0.8852
Epoch 4/100
25/25 [==============================] - 0s 4ms/step - loss: 0.3608 - accuracy: 0.8519 - val_loss: 0.4046 - val_accuracy: 0.8852
Epoch 5/100
25/25 [==============================] - 0s 3ms/step - loss: 0.3463 - accuracy: 0.8477 - val_loss: 0.4298 - val_accuracy: 0.8852
Epoch 6/100
25/25 [==============================] - 0s 4ms/step - loss: 0.3242 - accuracy: 0.8601 - val_loss: 0.4143 - val_accuracy: 0.8852
Epoch 7/100
25/25 [==============================] - 0s 4ms/step - loss: 0.3123 - accuracy: 0.8642 - val_loss: 0.4304 - val_accuracy: 0.8689
Epoch 8/100
25/25 [===

In [5]:
test_loss, test_acc = model.evaluate(X_test_prepared, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}, Test loss: {test_loss}')


3/3 - 0s - loss: 1.7081 - accuracy: 0.7792 - 30ms/epoch - 10ms/step

Test accuracy: 0.7792207598686218, Test loss: 1.7080605030059814


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_prepared.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Model summary to show architecture
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 128)               2688      
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dense_7 (Dense)             (None, 1)                 33        
                                                                 
Total params: 13057 (51.00 KB)
Trainable params: 13057 (51.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
